**Preprocessing Pipeline**

In [2]:
# Imports
import librosa
import numpy as np
import soundfile as sf
from scipy.signal import fftconvolve

import matplotlib.pyplot as plt
import soundfile as sf

**1. Synchronization of recorded audio files**

In [ ]:
# Load and synchronize audio files
print("Loading Pi1 audio file...")
audio_pi1, sr1 = librosa.load("Align FFT/pi1_audio.wav", sr=None)  # Update with your correct path
print("Pi1 audio loaded.")

print("Loading Pi2 audio file...")
audio_pi2, sr2 = librosa.load("Align FFT/pi2_audio.wav", sr=None)  # Update with your correct path
print("Pi2 audio loaded.")

In [ ]:
# Ensure the sample rates match
if sr1 != sr2:
    raise ValueError("Sample rates of the two files do not match")

# Trim silence
print("Trimming silence from both audio files...")
audio_pi1, _ = librosa.effects.trim(audio_pi1)
audio_pi2, _ = librosa.effects.trim(audio_pi2)

# Cross-correlation using FFT
print("Computing cross-correlation with FFT...")
correlation = fftconvolve(audio_pi1, audio_pi2[::-1], mode='full')
lag = np.argmax(correlation) - len(audio_pi2)
print(f"Detected delay: {lag / sr1} seconds")

# Align the audio files based on the lag
print("Aligning audio files...")
if lag > 0:
    aligned_pi1 = audio_pi1[lag:]
    aligned_pi2 = audio_pi2
else:
    aligned_pi1 = audio_pi1
    aligned_pi2 = audio_pi2[-lag:]

# Ensure both audio files are of the same length
min_length = min(len(aligned_pi1), len(aligned_pi2))
aligned_pi1 = aligned_pi1[:min_length]
aligned_pi2 = aligned_pi2[:min_length]

# Combine both audio files into a stereo format
print("Combining aligned audio into stereo format...")
combined_audio = np.vstack((aligned_pi1, aligned_pi2))

In [ ]:
# Save the aligned audio files
print("Saving aligned audio files...")
sf.write("Align FFT/aligned_pi1.wav", aligned_pi1, sr1)
sf.write("Align FFT/aligned_pi2.wav", aligned_pi2, sr1)

# Save a stereo version of the aligned audio
sf.write("Align FFT/aligned_stereo.wav", combined_audio.T, sr1)

print("Processing and saving completed.")

**2. Transformations (STFT & MEL)**

In [ ]:
# Paths to your audio files in the Align FFT folder
audio_pi1_path = "Align FFT/aligned_pi1.wav"
audio_pi2_path = "Align FFT/aligned_pi2.wav"

# Load aligned audio files
print("Loading aligned Pi1 audio file...")
aligned_pi1, sr1 = librosa.load(audio_pi1_path, sr=None)
print("Aligned Pi1 audio loaded.")

print("Loading aligned Pi2 audio file...")
aligned_pi2, sr2 = librosa.load(audio_pi2_path, sr=None)
print("Aligned Pi2 audio loaded.")


In [ ]:
# Apply STFT (Short-term Fourier Transform)
print("Applying Short-term Fourier Transform (STFT)...")
stft_pi1 = librosa.stft(aligned_pi1, n_fft=1024, hop_length=512)
stft_pi2 = librosa.stft(aligned_pi2, n_fft=1024, hop_length=512)

In [ ]:
# Convert to Mel-spectrogram
print("Converting to Mel-spectrogram...")
mel_pi1 = librosa.feature.melspectrogram(S=np.abs(stft_pi1), sr=sr1, n_mels=128)
mel_pi2 = librosa.feature.melspectrogram(S=np.abs(stft_pi2), sr=sr2, n_mels=128)

# Convert Mel-spectrogram to dB scale
print("Converting Mel-spectrogram to dB scale...")
mel_db_pi1 = librosa.power_to_db(mel_pi1, ref=np.max)
mel_db_pi2 = librosa.power_to_db(mel_pi2, ref=np.max)

# Normalize the mel-spectrograms
print("Normalizing Mel-spectrograms...")
mel_db_pi1 = (mel_db_pi1 - np.mean(mel_db_pi1)) / np.std(mel_db_pi1)
mel_db_pi2 = (mel_db_pi2 - np.mean(mel_db_pi2)) / np.std(mel_db_pi2)

In [ ]:
# Save Mel-spectrograms as numpy arrays for future use
print("Saving Mel-spectrograms...")
np.save("mel_db_pi1.npy", mel_db_pi1)
np.save("mel_db_pi2.npy", mel_db_pi2)


In [ ]:
# Plot and save the Mel-spectrograms
print("Plotting and saving Mel-spectrograms...")
plt.figure(figsize=(10, 4))
librosa.display.specshow(mel_db_pi1, sr=sr1, x_axis='time', y_axis='mel', cmap='viridis')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-Spectrogram Pi1')
plt.savefig('mel_spectrogram_pi1.png')
plt.show()

plt.figure(figsize=(10, 4))
librosa.display.specshow(mel_db_pi2, sr=sr2, x_axis='time', y_axis='mel', cmap='viridis')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-Spectrogram Pi2')
plt.savefig('mel_spectrogram_pi2.png')
plt.show()